In [4]:
import dotenv
import os
dotenv.load_dotenv()
import json

from pydantic import BaseModel
from typing import List, Dict, Tuple, Set

from google import genai
from google.genai import types
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch


class Affiliation(BaseModel):
  name: str
  title: str

class ContactInfo(BaseModel):
  email: str
  phone: str

class Researcher(BaseModel):
  name_in_Chinese: str
  research_keywords: List[str]
  affiliations: List[Affiliation]
  contact_info: ContactInfo

## Multi-agent approach

Gemini does not seem to be able to do many tasks at once. For example, it cannot consistently generate a JSON string in text-output mode -- cannot do JSON output when tools are used. Some instrctions are not properly followed, such as "do not tell me how you think but just give me the result". 

So let's create individual agents for each task. 

* Agent 1: Web search with Gemini's own query rewriting and thinking
* Agent 2: Check the quality of the web search results
* Agent 3: Generate the final JSON output
* Main function: coordinate the work of the three agents. Need to obtain at least three candidates or three consistent "I cannot find" responses.

In [45]:
class InfoGatherer:
  def __init__(self, model_id: str):
    self.system_prompt = "You are a helpful cross-lingual customer discovery agent."

    self.user_prompt = """
    ## Background and overview

    Many researchers who are China-based or have affiliations in China publish papers in English. On those papers, their names are not in Chinese characters. For example, their names are transliterated or romanized, such as in Pinyin or Wade-Giles. The goal here is to find their names in Chinese characters. Optionally, find their affiliations in Chinese characters, and their emails and phone numbers.
    
    ### Challenges
    
    1. Mapping from such transliterated/romanized names to their Chinese names is not a simple machine translation task due to the homophony of Chinese characters and different transliteration/romanization methods. There is no one-to-one mapping. 
    2. Many researchers use names in the English-speaking world that are not a direct transliteration or romanization of their Chinese name. For example, nvidia CEO Jensen Huang's Chinese name is 黃仁勳 whose romanization is Jen-Hsun Huang. As another example, Forrest Bao's Chinese given name has nothing to do with and cannot be transliterated to "Forrest". So do not rule out a name that does not match the transliterated/romanized name.
    3. Reseachers move from one affiliation to another. So do not rule out a researcher that does not match the affiliation provided in the input. Two researchers may be the same person if they share co-authors, especially when the co-authors are from the same institutions at the two occurences of the researcher.
    4. It is possible for a researcher to have multiple affiliations, some in China and some in other countries. S/he can have multiple affliations in China and multiple affliations in other countries. S/he may have multiple affiliations on one paper. So do not rely solely on the affiliation provided in the input.

    ### Strategies

    Please iteratively search the web. First, come up with an initial search query. Then, search the web. After each round, you should analyze the results, then think, and then refine or expand the search query for the next round. Stop when you are confident that you have found the researcher in the Chinese-speaking world or no new results can be found.

    Here are some ideas to build or expand the search queries but please do not limit yourself to these ideas:

    {strategies}

    ### Knowledge about Chinese names that may help you

    1. While there is no one-to-one mapping from transliterated/romanized names to Chinese names, the mapping of last/family Chinese names are more finite than the first/given names. For example, if you see "Huang" or "Wong" at the beginning or the end of someone's name, it is highly likely to be the last/family name and the character is almost certain to be 黄, 汪, 翁, or 王.
    2. In the transliteration/romanization of a Chinese name, the last/family name usually appear as the first word or the last word -- either case is common. A word or token in the middle of a transliterated/romanized Chinese name is mostly unlikely to be a last/family name.
    3. If someone's name in the English-speaking world is in the form of "a Western name + a Chinese name", the Chinese name is most likely the last/family name.
    4. A Chinese last name is usually a single character or syllable.

    ### Qualification

    1. You can search in both English and Chinese resources, but the Chinese name of a researcher must appear in Chinese sources (can be part of an English-dominant resources) to support the name and affiliation of the researcher.
    2. It is possible that no information in Chinese can support the name and affiliation of the researcher. In such case, you should say I don't know. Do not use results of researchers of similar names.
    3. There is no misspelling of the name or affiliation provided in the input. 

    ### Output format

    {output_format}

    Now, lets begin! Below is the information of the researcher to find:
    {researcher_info}
    """

    self.output_format_markdown = """
    
    Output the name and affiliation of the researcher in the following markdown format:
    * Chinese name 1
    * Affiliation 1
      - quote in Chinese source
      - title (if possible)
      - email (if possible)
      - phone number (if possible)
    * Affiliation 2
      - quote in Chinese source
      - title (if possible)
      - email (if possible)
      - phone number (if possible)
    ...

    For each name and affiliation, please provide a quote from the Chinese source to support it.   

    """

    # Tell me all information you found on the internet and how you processed it. 

    # At the end, 


    self.output_format_json = """

    Output the name and affiliation of the researcher in the following json format:

    {{
      "name": "Name of the researcher",
      "affiliations": [
        {{
          "name": "Affiliation name",
          "title": "Title of the researcher in this affiliation (if possible)",
          "quote": "Quote from the Chinese source showing this researcher is employed/affiliated with this institution",
          "email": "Email address of the researcher (if possible)",
          "phone_number": "Phone number of the researcher (if possible)"
        }}
      ]
    }}
    """

    self.strategies = """
    * See if the research info includings affiliation. If so, translate the affiliation to Chinese and add it to the search queries. 
    * Most scientific publications include the email address and phone number of authors. If you don't have the email address or phone number of the researcher, you may prioritize your search sources to scientific journals, either in English or Chinese, hoping to find the email address or phone number of the researcher. Once you have the email address or phone number of the researcher, you can add them to search query and search on Chinese resources especially scientific journals where the Chinese name of the researcher is most likely to be found. 
    * Different Chinese names can be transliterated or Romanized to the same name in English. Please try to come up some possible Chinese names, and then use them as part of the search terms. Doing so for the family/last name is easier than the first name. 
    * Prioritize your search sources to Chinese-speaking websites. Also try to search as much as possible from scientific literature sources such as Google Scholar, PubMed, CNKI, Semantic Scholar, etc.
    * Determine whether the name in English is romanized or transliterated, in the methods including but not limited to Pinyin, Wade-Giles, etc. If the name is romanized, confine your thinking and next round of search to Chinese names that match the romanized version.
    * Leverage the co-author network/circle of the researcher. You may find co-author network from Semantic Scholar, Google Scholar, etc.
    * A research may publish papers under multiple affiliations. Expand your search queries to include other affiliations appear in publications as long as they are related to the research topic. 
    * From all pages you have found, extract all names and affiliations and add them to the search queries. 
    """

    self.model_id = model_id

  def call_gemini(self, prompt):
    client = genai.Client()

    google_search_tool = Tool(
        google_search = GoogleSearch()
    )

    response = client.models.generate_content(
        model=self.model_id,
        contents=prompt,
        config= {
           "tools": [google_search_tool],
           "response_modalities": ["TEXT"],
           "thinking_config": types.ThinkingConfig(thinking_budget=-1), 
           "system_instruction": self.system_prompt
        }
    )
    return response

  def calculate_cost(self, response):
    web_search_queries = response.candidates[0].grounding_metadata.web_search_queries
    
    pricing = { # per 1M tokens
      "gemini-2.5-flash": 
        {"input": 0.30, "output": 2.5, "cache": 0.075, "web_search": 0.035},
      "gemini-2.5-pro":
        {"input": 1.25, "output": 10, "cache": 0.31, "web_search": 0.035},
    }

    model_id = self.model_id

    pricing_for_model = pricing[model_id]

    cost = 0
    cost += pricing_for_model["input"] * response.usage_metadata.prompt_token_count / 1000000
    cost += pricing_for_model["output"] * (response.usage_metadata.candidates_token_count + response.usage_metadata.thoughts_token_count) / 1000000
    num_cached_tokens = 0 if response.usage_metadata.cached_content_token_count is None else response.usage_metadata.cached_content_token_count
    cost += pricing_for_model["cache"] * num_cached_tokens / 1000000
    cost += pricing_for_model["web_search"] * len(web_search_queries) / 1000
    
    return cost

  def one_prompt_approach(self, researcher_info: str, output_format: str) -> Tuple[List[str], float]:

    if output_format == "json":
      output_format_prompt = self.output_format_json
    elif output_format == "markdown":
      output_format_prompt = self.output_format_markdown
    else:
      raise ValueError(f"Invalid output format: {output_format}")

    prompt = self.user_prompt.format(researcher_info=researcher_info, strategies=self.strategies, output_format=output_format_prompt)

    response = self.call_gemini(prompt)
 
    try:
      thoughts = [part.text for part in response.candidates[0].content.parts]
    except:
      print ("Error: in candidate.content.parts. ")
      thoughts = []

    cost = self.calculate_cost(response)

    # the_other_model_id = {"gemini-2.5-flash": "gemini-2.5-pro", "gemini-2.5-pro": "gemini-2.5-flash"}[self.model_id]
    # print (f"Cost of using {model_id}: ${cost:.6f}")
    # print (f"The cost would have been ${calculate_cost(response, the_other_model_id):.6f} if using {the_other_model_id}")
    
    return thoughts, cost

In [46]:
# Try again on another researcher
info_gatherer = InfoGatherer(model_id="gemini-2.5-flash")
researcher_info = "Yifeng Wang College of Materials Science and Engineering, NanjLin Tech University, Nanjing, Jiangsu, 210009, China"
thoughts, cost = info_gatherer.one_prompt_approach(researcher_info=researcher_info, output_format="json")
print("".join(thoughts), cost)

```json
{
  "name": "王一峰",
  "affiliations": [
    {
      "name": "南京工业大学 材料科学与工程学院",
      "title": "副教授",
      "quote": "王一峰. 副教授 南京工业大学 材料科学与工程学院. [4]",
      "email": null,
      "phone_number": "025-83587260"
    }
  ]
}
``` 0.012161775000000001


In [47]:
# Try again on another researcher
info_gatherer = InfoGatherer(model_id="gemini-2.5-flash")
thoughts, cost = info_gatherer.one_prompt_approach("Professor David Z. Zhu, Department of Civil and Environmental Engineering, University of Alberta, Edmonton, AB, T6G 1H9, Canada", output_format="json")
print("".join(thoughts))

```json
{
  "name": "朱志伟 (Zhu Zhiwei)",
  "affiliations": [
    {
      "name": "宁波大学 (Ningbo University)",
      "title": "水利与海洋工程研究院院长 (Dean of the Institute of Hydraulic and Ocean Engineering), 教授 (Professor)",
      "quote": "加拿大工程院院士、学校水利与海洋工程研究院院长朱志伟获国际科学技术合作奖 [4]",
      "email": "david.zhu@ualberta.ca",
      "phone_number": null
    },
    {
      "name": "University of Alberta",
      "title": "Professor Emeritus, Department of Civil and Environmental Engineering",
      "quote": "Dr. David Zhu is a Professor at Ningbo University in China. He is a Professor Emeritus in the Department of Civil and Environmental Engineering at the University of Alberta, Canada where he was a faculty member for 25 years. [7]",
      "email": "david.zhu@ualberta.ca",
      "phone_number": null
    }
  ]
}
```


In [48]:
# try Gemini Flash with the InfoGatherer
info_gatherer = InfoGatherer(model_id="gemini-2.5-flash")
researcher_info = """gu	aijuan	Photocatalysis,Oxides,Adsorption,Organic reactions,Catalytic activity"""
thoughts, cost = info_gatherer.one_prompt_approach(researcher_info, output_format="json")
print ("".join(thoughts))
print ("Cost: ", cost)

```json
{
  "name": "顾嫒娟",
  "affiliations": [
    {
      "name": "苏州大学材料与化学化工学部",
      "title": "硕士研究生导师",
      "quote": "顾嫒娟列于苏州大学材料与化学化工学部材料科学与工程（材料学）硕士研究生导师名单中. [7]",
      "email": "ajgu@suda.edu.cn",
      "phone_number": "+86 512 65880967"
    },
    {
      "name": "Jiangsu Key Laboratory of Advanced Functional Polymer Design and Application, Department of Materials Science & Engineering, College of Chemistry, Chemical Engineering and Materials Science, Soochow University",
      "title": "Corresponding author",
      "quote": "Xiaoling Zhu , Li Yuan , Guozheng Liang * and Aijuan Gu * Jiangsu Key Laboratory of Advanced Functional Polymer Design and Application, Department of Materials Science & Engineering, College of Chemistry, Chemical Engineering and Materials Science, Soochow University, Suzhou 215123, China. E-mail: ajgu@suda.edu.cn; lgzheng@suda.edu.cn; Fax: +86 512 65880089; Tel: +86 512 65880967. [5, 6]",
      "email": "ajgu@suda.edu.cn",
      "phone_number": "+86 

In [ ]:
class RomanizationWorker():
    def __init__(self):
        self.system_prompt = "You are a helpful language expert."

    def detect(self, name: str) -> bool:
        """Determine if a name is romanized or transliterated from Chinese characters."""
        client = genai.Client()
        prompt = """Is the name below romanized or transliterated from Chinese characters? The romanization method can be Pinyin, Wade-Giles, Jyutping, Yale, or other methods. Return only "true" or "false". The romanized/transliterated name is: {name}"""

        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt.format(name=name),
            config= {
                "response_modalities": ["TEXT"],
                "thinking_config": types.ThinkingConfig(thinking_budget=-1), 
                "system_instruction": self.system_prompt
            }
        )

        try:
            thoughts = [part.text for part in response.candidates[0].content.parts]
        except:
            print ("Error: in candidate.content.parts. ")
            thoughts = ["I don't know."]

        thoughts = "\n".join(thoughts)

        return thoughts

    def Romanization_and_Chinese_Character_matcher(self, query: str, Chinese_name: str) -> bool:
        """Given a _name_ in Chinese characters, check whether the romanized/transliterated name is part of the _query_"""
        client = genai.Client()
        # TODO: ffinish the prompt
        prompt = """ Return only "true" or "false". The romanized/transliterated name is: {name}"""

        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt.format(name=name),
            config= {
                "response_modalities": ["TEXT"],
                "thinking_config": types.ThinkingConfig(thinking_budget=-1), 
                "system_instruction": self.system_prompt
            }
        )

        try:
            thoughts = [part.text for part in response.candidates[0].content.parts]
        except:
            print ("Error: in candidate.content.parts. ")
            thoughts = ["I don't know."]

        thoughts = "\n".join(thoughts)

        return thoughts

romanization_detector = Romanization_Detector()
romanization_detector.detect("Jen-Hsun Huang")   
    

'true'

In [ ]:
class InfoGatherWithTools(InfoGatherer):
    def __init__(self, model_id: str):
        super().__init__(model_id)
        

In [ ]:
class JSONGenerator:
  def __init__(self):
    self.system_prompt = """You are a helpful assistant to extract and rectify JSON strings."""
    self.user_prompt = """Given a string that may contain JSON string, output synatically correct JSON string that match the provided schema. The provided string may or may not contain a JSON string. If it does not contain a JSON string, extract information from the string and output a JSON string that match the provided schema. The JSON string in the input may be corrupted. If it is, rectify it to make it synatically correct.     
    """
    # TODO: Finish and test it 

  def call_gemini(self, query: str, thoughts: List[str]) -> str:
    client = genai.Client()
    prompt = self.user_prompt.format(query=query, thoughts=thoughts)

    response = client.models.generate_content(
      model="gemini-2.5-flash",
      contents=prompt,
      config= {
          # "response_modalities": ["TEXT"],
          # "thinking_config": types.ThinkingConfig(thinking_budget=-1), 
          "system_instruction": self.system_prompt,
          "response_mime_type": "application/json",
          "response_schema": Researcher
      }
    )
    researcher = json.loads(response.text)
    return researcher


In [ ]:
info_gatherer = InfoGatherer(model_id="gemini-2.5-flash")
json_generator = JSONGenerator()

query = "hua chen	jian	Palladium,Layers,Conjugated polymers,Light,Perovskites"
thoughts, cost = info_gatherer.one_prompt_approach(query)
r = json_generator.call_gemini(query, thoughts)
print (json.dumps(r, indent=2, ensure_ascii=False))

Error: in candidate.content.parts. 
{
  "name_in_Chinese": "",
  "research_keywords": [],
  "affiliations": [],
  "contact_info": {
    "email": "",
    "phone": ""
  }
}


In [ ]:
# try again on another researcher
query = "Yifeng Wang College of Materials Science and Engineering, Nanjin Tech University, Nanjing, Jiangsu, 210009, China"
thoughts, cost = info_gatherer.one_prompt_approach(query)
r = json_generator.call_gemini(query, thoughts)
print (json.dumps(r, indent=2, ensure_ascii=False))

{
  "name_in_Chinese": "王一峰",
  "research_keywords": [],
  "affiliations": [
    {
      "name": "南京工业大学 材料科学与工程学院",
      "title": "教授, 博士生导师"
    }
  ],
  "contact_info": {
    "email": "yifeng.wang@njtech.edu.cn",
    "phone": ""
  }
}
